In [1]:
game_states = {
    "102012100": {
        "possible_moves": {
            "112012100": {"p_win": 0.01},
            "102112100": {"p_win": 0.02},
            "102012110": {"p_win": 0.03},
            "102012101": {"p_win": 0.04},
        }
    }
}

next_move_options = game_states["102012100"]["possible_moves"]
next_move_options

{'112012100': {'p_win': 0.01},
 '102112100': {'p_win': 0.02},
 '102012110': {'p_win': 0.03},
 '102012101': {'p_win': 0.04}}

In [2]:
def winning_state(game_state: str, player: str = "1") -> bool:
    """Check the game state to see if it indicates a winning state."""
    # check rows
    if game_state[0] == game_state[1] == game_state[2] == player:
        return True
    if game_state[3] == game_state[4] == game_state[5] == player:
        return True
    if game_state[6] == game_state[7] == game_state[8] == player:
        return True

    # check columns
    if game_state[0] == game_state[3] == game_state[6] == player:
        return True
    if game_state[1] == game_state[4] == game_state[7] == player:
        return True
    if game_state[2] == game_state[5] == game_state[8] == player:
        return True

    # check diagonals
    if game_state[0] == game_state[4] == game_state[8] == player:
        return True
    if game_state[2] == game_state[4] == game_state[6] == player:
        return True

    # otherwise...
    return False


print(f"111212122: win for Player One --> {winning_state('111212122', player='1')}")
print(f"120120100: win for Player One --> {winning_state('111212100', player='1')}")
print(f"120210001: win for Player One --> {winning_state('120210001', player='1')}")
print(f"201210100: win for Player One --> {winning_state('120210001', player='1')}")
print(f"112012100: win for Player One --> {winning_state('112012100', player='1')}")
print(f"222121211: win for Player Two --> {winning_state('222121211', player='2')}")
print(f"112102002: win for Player Two --> {winning_state('112102002', player='2')}")

111212122: win for Player One --> True
120120100: win for Player One --> True
120210001: win for Player One --> True
201210100: win for Player One --> True
112012100: win for Player One --> False
222121211: win for Player Two --> True
112102002: win for Player Two --> True


In [3]:
def current_player(game_state: str) -> str:
    """Get current player based on game state."""
    return "1" if game_state.count("1") <= game_state.count("2") else "2"


def get_possible_moves(game_state: str) -> list[str]:
    """Compute all possible next moves."""
    possible_moves: list[str] = []
    for i in range(9):
        if game_state[i] == "0":
            possible_moves.append(
                game_state[:i] + current_player(game_state) + game_state[i+1:]
            )
    return possible_moves


get_possible_moves("120120100")

['122120100', '120122100', '120120120', '120120102']

In [4]:
possible_games: list[list[str]] = []
possible_games.append(["000000000"])

# phase 1 - generate all possible games
for game_round in range(9):
    new_possible_games: list[list[str]] = []
    for game in possible_games:
        previous_game_state = game[-1]
        if (winning_state(previous_game_state, player="1")
            or winning_state(previous_game_state, player="2")):
            new_possible_games.append(game.copy())
            continue
        for move in get_possible_moves(previous_game_state):
            new_game = game.copy()
            new_game.append(move)
            new_possible_games.append(new_game)
        possible_games = new_possible_games

print(f"number of possible games = {len(possible_games):,}")

number of possible games = 255,168


In [5]:
def print_final_game_state(game_path: list[str]) -> None:
    """Print the game state on a 3 x 3 board."""
    final_game_state = game_path[-1]
    print(
        "final game state:" + "\n "
        + final_game_state[0:3] + "\n "
        + final_game_state[3:6] + "\n "
        + final_game_state[6:9]
    )


def print_game_path(game_path: list[str]) -> None:
    """Print the path of a finished game."""
    all_states_in_path = [state for state in game_path]
    print("game path: " + " --> ".join(all_states_in_path))


def print_game_result(game_path: list[str]) -> None:
    """Print the game result."""
    final_fame_state = game_path[-1]
    if winning_state(final_fame_state, "1"):
        print("result: player 1 wins")
    elif winning_state(final_fame_state, "2"):
        print("result: player 2 wins")
    else:
        print("result: draw")


game_path = 130
print_game_result(possible_games[game_path])
print_game_path(possible_games[game_path])
print_final_game_state(possible_games[game_path])

result: draw
game path: 000000000 --> 100000000 --> 120000000 --> 121000000 --> 121020000 --> 121021000 --> 121021002 --> 121121002 --> 121121202 --> 121121212
final game state:
 121
 121
 212


In [6]:
def game_outcome(game_path: list[str]) -> str:
    """Compute the game's outcome."""
    final_fame_state = game_path[-1]
    if winning_state(final_fame_state, "1"):
        return ("player 1 wins")
    elif winning_state(final_fame_state, "2"):
        return ("player 2 wins")
    else:
        return("draw")


num_player_1_wins = sum(
    game_outcome(game) == "player 1 wins" for game in possible_games
)
num_player_2_wins = sum(
    game_outcome(game) == "player 2 wins" for game in possible_games
)
num_draws = sum(
    game_outcome(game) == "draw" for game in possible_games
)

print(f"number of times player 1 wins = {num_player_1_wins:,}")
print(f"number of times player 2 wins = {num_player_2_wins:,}")
print(f"number of draws = {num_draws:,}")

number of times player 1 wins = 131,184
number of times player 2 wins = 77,904
number of draws = 46,080


In [7]:
from collections import Counter

p1_win_counter: Counter[str] = Counter()
p2_win_counter: Counter[str]= Counter()
draw_counter: Counter[str] = Counter()
interim_states: set[str] = set()

outcome_probabilities: dict[str, dict[str, float]] = {}
ai_player_database: dict[str, dict[str, dict[str, float]]] = {}

# count wins and draws for every interim state in every game
for game in possible_games:
    final_game_state = game[-1]
    for game_state in game:  ## game[:-2] or game[:-1] analysis?
        interim_states.add(game_state)
        if winning_state(final_game_state, player="1"):
            p1_win_counter.update([game_state])
        elif winning_state(final_game_state, player="2"):
            p2_win_counter.update([game_state])
        else:
            draw_counter.update([game_state])

# compute outcome probabilities from every possible interim state
for game_state in interim_states:
    num_p1_wins_from_state = p1_win_counter[game_state]
    num_p2_wins_from_state = p2_win_counter[game_state]
    num_draws_from_state = draw_counter[game_state]

    num_final_outcomes = (
        num_p1_wins_from_state + num_p2_wins_from_state + num_draws_from_state
    )

    outcome_probabilities[game_state] = {
        "p1_wins": round(num_p1_wins_from_state / num_final_outcomes, 5),
        "p2_wins": round(num_p2_wins_from_state / num_final_outcomes, 5),
        "draw": round(num_draws_from_state / num_final_outcomes, 5)
    }

# compute data in a form that can be used for an AI player
for game_state in interim_states:
    if winning_state(game_state, player="1") or winning_state(game_state, player="2"):
        continue
    possible_moves_data: dict[str, dict[str, float]] = {}
    for possible_move in get_possible_moves(game_state):
        possible_moves_data[possible_move] = outcome_probabilities[possible_move]
    ai_player_database[game_state] = possible_moves_data

In [8]:
from json import dumps

current_state = "000000000"
for possible_move, data in ai_player_database[current_state].items():
    print(f"{current_state} --> {possible_move}: {dumps(data)}")

000000000 --> 100000000: {"p1_wins": 0.52834, "p2_wins": 0.28473, "draw": 0.18693}
000000000 --> 010000000: {"p1_wins": 0.48094, "p2_wins": 0.34388, "draw": 0.17518}
000000000 --> 001000000: {"p1_wins": 0.52834, "p2_wins": 0.28473, "draw": 0.18693}
000000000 --> 000100000: {"p1_wins": 0.48094, "p2_wins": 0.34388, "draw": 0.17518}
000000000 --> 000010000: {"p1_wins": 0.60482, "p2_wins": 0.21707, "draw": 0.17811}
000000000 --> 000001000: {"p1_wins": 0.48094, "p2_wins": 0.34388, "draw": 0.17518}
000000000 --> 000000100: {"p1_wins": 0.52834, "p2_wins": 0.28473, "draw": 0.18693}
000000000 --> 000000010: {"p1_wins": 0.48094, "p2_wins": 0.34388, "draw": 0.17518}
000000000 --> 000000001: {"p1_wins": 0.52834, "p2_wins": 0.28473, "draw": 0.18693}


In [9]:
def next_best_move(game_state: str, strategy: str = "win") -> str:
    """Suggest the best next move."""
    player = current_player(game_state)
    possible_move_data = ai_player_database[game_state]

    if strategy == "win":
        data_key = f"p{player}_wins"
        moves_and_probs: list[tuple[str, float]] = []
        for move, move_data in possible_move_data.items():
            moves_and_probs.append((move, move_data[data_key]))
        moves_sorted = sorted(moves_and_probs, key=lambda e: e[1])
        best_move = moves_sorted[-1]

    print(f"{game_state} --> {best_move[0]}")
    return best_move[0]

next_best_move("000000000")

000000000 --> 000010000


'000010000'

In [10]:
current_state = "000000000"
for round in range(9):
    player = current_player(current_state)
    current_state = next_best_move(current_state)
    if winning_state(current_state, player):
        print(f"player {player} wins!")
        break
    if round == 9:
        print("draw!")

000000000 --> 000010000
000010000 --> 000010002
000010002 --> 000010102
000010102 --> 000012102
000012102 --> 001012102
player 1 wins!


In [11]:
import json

with open("ai_db.json", "w") as f:
    json.dump(ai_player_database, f)
